# Tuning a Keras Model with `kerastuner`

In [55]:
import keras

#dataset = keras.datasets.mnist
#dataset = keras.datasets.fashion_mnist

# Boston housing: 404 data points: (x,y) = (x1 ... x13, y) where x1 thru x13 are parameters like
# locale, crime rate, prop tax, and y = pricing.
dataset = keras.datasets.boston_housing
(x_train, y_train), (x_val, y_val) = dataset.load_data()

print(f"Dataset => {dataset}"
      f"\nshapes (x,y, x test, ytest) => {x_train.shape}, {y_train.shape}, {x_val.shape}, {y_val.shape}"
      f"\ndata types => {x_train.dtype}, {y_train.dtype}, {x_val.dtype}, {y_val.dtype}")

Dataset => <module 'keras.api.datasets.boston_housing' from '/Users/jcoles/Source/jkc/jims-ml-sandbox/.venv/lib/python3.10/site-packages/keras/api/datasets/boston_housing/__init__.py'>
shapes (x,y, x test, ytest) => (404, 13), (404,), (102, 13), (102,)
data types => float64, float64, float64, float64


In [68]:
from keras.src.layers import Dense
from keras import layers, Model, optimizers
from keras import Sequential
from keras_tuner import HyperModel


# class MyHyperModel(HyperModel):
#     def build(self, hp):
#         model = Sequential()
#         model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
#                         activation=hp.Choice('activation', ['relu', 'tanh'])))
#         model.add(Dense(10, activation='softmax'))
#         model.compile(optimizer=hp.Choice('optimizer', ['adam', 'sgd']),
#                       loss='sparse_categorical_crossentropy',
#                       metrics=['accuracy'])
#         return model


def build_mlp1_model(hp):
    #    inputs = layers.Input(shape=(784,))
    inputs = layers.Input(shape=(13,))
    x = layers.Dense(units=hp.Int('units', min_value=16, max_value=64, step=16),
                     activation=hp.Choice('activation', ['relu', 'sigmoid']))(inputs)
    outputs = layers.Dense(1)(x)
    model = Model(inputs, outputs)
    optimizer = hp.Choice("optimizer", values=["adam", "rmsprop", "sgd"])
    model.compile(optimizer=optimizer,
                  loss='mse',
                  metrics=['mse']
                  )
    return model


In [69]:
from kerastuner.tuners import RandomSearch

tuner = RandomSearch(
    build_mlp1_model,
    objective='val_mse',  # Specify the metric to optimize
    max_trials=10,  # Number of models to try
    executions_per_trial=2,  # Average over 2 runs for each hyperparameter set
    directory='my_dir',  # Directory to save search results
    project_name='boston_tuning_example',  # Project name for organization
    overwrite=True
)


In [70]:
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))


Trial 10 Complete [00h 00m 03s]
val_mse: 341.1876525878906

Best val_mse So Far: 63.784671783447266
Total elapsed time: 00h 00m 25s


In [71]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Optimal number of units in Dense layer: {best_hps.get('units')}")
print(f"Optimal activation function: {best_hps.get('activation')}")
print(f"Optimal optimizer: {best_hps.get('optimizer')}")

# Build the best model and re-train if necessary
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(x_train, y_train, epochs=50, validation_split=0.2)


Optimal number of units in Dense layer: 32
Optimal activation function: relu
Optimal optimizer: rmsprop
Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 12502.6504 - mse: 12502.6504 - val_loss: 3834.6177 - val_mse: 3834.6177
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3215.4004 - mse: 3215.4004 - val_loss: 1024.4263 - val_mse: 1024.4263
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 876.7130 - mse: 876.7130 - val_loss: 384.0189 - val_mse: 384.0189
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 379.5747 - mse: 379.5747 - val_loss: 290.1783 - val_mse: 290.1783
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 317.7613 - mse: 317.7613 - val_loss: 245.2735 - val_mse: 245.2735
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 255.5800 - mse: 255.5800 - val_loss: 216.3833 - val_mse: 216.3833
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 204.6380 - mse: 204.6380 - val_loss: 181.1153 - val_mse: 181.1153
Epoch 8/50
11/11 

In [72]:
tuner.results_summary()


Results summary
Results in my_dir/boston_tuning_example
Showing 10 best trials
Objective(name="val_mse", direction="min")

Trial 02 summary
Hyperparameters:
units: 32
activation: relu
optimizer: rmsprop
Score: 63.784671783447266

Trial 07 summary
Hyperparameters:
units: 64
activation: relu
optimizer: rmsprop
Score: 70.82283782958984

Trial 01 summary
Hyperparameters:
units: 48
activation: relu
optimizer: rmsprop
Score: 82.92798233032227

Trial 08 summary
Hyperparameters:
units: 64
activation: relu
optimizer: adam
Score: 84.91437911987305

Trial 03 summary
Hyperparameters:
units: 32
activation: relu
optimizer: adam
Score: 107.76593780517578

Trial 06 summary
Hyperparameters:
units: 16
activation: relu
optimizer: rmsprop
Score: 121.0833625793457

Trial 04 summary
Hyperparameters:
units: 64
activation: sigmoid
optimizer: adam
Score: 337.6020812988281

Trial 09 summary
Hyperparameters:
units: 16
activation: relu
optimizer: adam
Score: 341.1876525878906

Trial 00 summary
Hyperparameters:
un